In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Standard library imports.
import os
from pathlib import Path
import itertools
import json
import pickle

# Related third party imports.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use(['science','notebook'])
plt.rcParams['figure.figsize'] = (12,6)

# Local application/library specific imports.
import src.models.vSDN_request
import src.models.metrics as metrics

In [ ]:
def preprocess_vSDN(vSDN):
    base_dict = vSDN.__dict__
    base_dict['size'] = vSDN.get_size()
    base_dict['revenue'] = metrics.get_revenue(vSDN)
    base_dict['util'] = metrics.get_utilization(vSDN)
    return base_dict

def calc_objective(df, metric: str = None):
    objectives = {}
    for t, dfg in df.groupby(by=['sim_dynamic_type', 'simulation_id', 'simulation_round']):
        objectives[t] = np.zeros(max(df['end_time']))
        # print(t, len(dfg))
        for _, vSDN in dfg.iterrows():
            if vSDN['accepted'] and vSDN['active_time'] == vSDN['TTL']:
                objectives[t][vSDN['end_time']-1] += vSDN[metric]
    return objectives

In [ ]:
obj = calc_objective(dfh, 'util')

fig, ax = plt.subplots(nrows=int(len(obj)/3), figsize=(8, 12))

for key, values in obj.items():
    ax[key[2]].plot(np.cumsum(values), label=f"{key[0]}-{key[2]}")
    ax[key[2]].legend()
    ax[key[2]].grid()

### Load vSDN history

In [ ]:
simulation_group_folder = '../results/25_italy/dynamic/2022-11-17-22-19-42'
simulation_group_id = os.path.basename(simulation_group_folder)

history = []
vSDN_history = {}

for filename in list(Path(simulation_group_folder).glob('*/history.p')):
    settings = json.load(open(filename.parent / 'settings.json', 'rb'))
    sim_history = pickle.load(open(filename, 'rb'))

    for sim_round, round_history in sim_history.items():
        vSDN_history[(settings['dynamic_type'], sim_round, settings['simulation_id'])] = round_history

        df = pd.DataFrame([preprocess_vSDN(vSDN) for vSDN in round_history.values()])
        df.insert(0, 'simulation_round', sim_round)
        df.insert(0, 'simulation_id', settings['simulation_id'])
        df.insert(0, 'sim_dynamic_type', settings['dynamic_type'])
        history.append(df)

dfh = pd.concat(history, axis=0).reset_index(drop=True)

In [ ]:
dfh.columns

In [ ]:
vSDN_history.keys()

In [ ]:
def calc_objective(vSDN_history, metric: str = None):
    objectives = {}
    for key, round_history in vSDN_history.items():
        objectives[key] = np.zeros(max(df['end_time']))

        for _, vSDN in round_history.items():
            if vSDN['accepted'] and vSDN['active_time'] == vSDN['TTL']:
                objectives[t][vSDN['end_time']-1] += vSDN[metric]
    return objectives

### Deployed vSDNs

In [ ]:
dfh.query("sim_dynamic_type == 'conservative' & accepted == True")

### Conservative - Liberal differences

In [ ]:
idc = dfh.query("dynamic_type == 'conservative'").accepted.to_numpy()
idl = dfh.query("dynamic_type == 'liberal'").accepted.to_numpy()
print(f"Conservative: {sum(idc)}\nLiberal: {sum(idl)}")

In [ ]:
dfh.query("dynamic_type == 'conservative'")[np.]

In [ ]:
dfl.iloc[idl.difference(idc)]

In [ ]:
# Discarded requests before TTL
violations = dfh.query('accepted & active_time != TTL')#['simulation_round'].value_counts()
violations

In [ ]:
dfh.query('accepted & dynamic_type == "liberal"')

In [ ]:
violations.simulation_round.value_counts()

In [ ]:
sns.countplot(violations.TTL - violations.active_time)